In [1]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer

# unet for imagen

unet1 = Unet(
    # dim = 128,
    # dim_mults = (1, 2, 4, 8),
    # num_resnet_blocks = 3,
    # layer_attns = (False, True, True, True),
    attn_dim_head = 64,
    attn_heads = 8,
    attn_pool_num_latents = 32,
    attn_pool_text = False,
    cond_on_text = True,
    dim_mults = [1, 2, 3, 4],
    dim = 192,
    ff_mult = 2.0,
    final_conv_kernel_size = 3,
    final_resnet_block = True,
    init_conv_to_final_conv_residual = False,
    layer_attns = [False, True, True, True],
    layer_cross_attns = [False, True, True, True],
    learned_sinu_pos_emb = True,
    lowres_cond = False,
    memory_efficient = False,
    num_resnet_blocks = 2,
    resnet_groups = 8,
    scale_resnet_skip_connection = False,
    use_global_context_attn = False,
    use_linear_attn = False,
)

# unet2 = Unet(
#     dim = 128,
#     cond_dim = 512,
#     dim_mults = (1, 2, 4, 8),
#     num_resnet_blocks = (2, 4, 8, 8),
#     layer_attns = (False, False, False, True),
#     layer_cross_attns = (False, False, False, True)
# )

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    unets = (unet1,),#, unet2),
    text_encoder_name = 't5-large',
    image_sizes = (64,),#, 256),
    timesteps = 1000,
    cond_drop_prob = 0.1
)#.cuda()

# wrap imagen with the trainer class

trainer = ImagenTrainer(imagen)

# mock images (get a lot of this) and text encodings from large T5

text_embeds = torch.randn(64, 256, 1024)#.cuda()
text_masks = torch.ones(64, 256).bool()#.cuda()
images = torch.randn(64, 3, 256, 256)#.cuda()

# feed images into imagen, training each unet in the cascade

for i in (1, 2):
    loss = trainer(
        images,
        text_embeds = text_embeds,
        text_masks = text_masks,
        unet_number = i,
        max_batch_size = 4        # auto divide the batch of 64 up into batch size of 4 and accumulate gradients, so it all fits in memory
    )

    trainer.update(unet_number = i)

# do the above for many many many many steps
# now you can sample an image based on the text embeddings from the cascading ddpm

images = trainer.sample(texts = [
    'a puppy looking anxiously at a giant donut on the table',
    'the milky way galaxy in the style of monet'
], cond_scale = 3.)

images.shape # (2, 3, 256, 256)

AssertionError: 